# RecSys
Modelo do tipo recuperação

## Importando bibliotecas necessárias

In [1]:
from sklearn import model_selection, metrics, preprocessing
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
import numpy as np
import torch

Configurando dispositivo para utilizar GPU se possível; do contrário, CPU

In [6]:
mode = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(mode)
device

device(type='cuda')

## Carregando dados do dataset de avaliações de filmes

In [16]:
df_movie_lens = pd.read_csv('data/movie-lens-small/ratings.csv')
df_movie_lens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [17]:
df_movie_lens.sample(5)

,userId,movieId,rating,timestamp
21663,140,37727,3.5,1145991113
47070,307,2325,2.5,1186172865
92323,597,1265,5.0,941557914
84439,542,1721,3.0,1163387005
36675,249,4975,3.5,1353276810


Contando quantidade usuários que avaliaram filmes

In [18]:
df_movie_lens['userId'].nunique()

610

Contando quantidade de filmes avaliados por usuários 

In [19]:
df_movie_lens['movieId'].nunique()

9724

Checando distribuição de avaliaçãoes (de 0 a 5) em porcentagem

In [26]:
df_movie_lens['rating'].value_counts(normalize=True) * 100

rating
4.0    26.595660
3.0    19.880797
5.0    13.101472
3.5    13.027093
4.5     8.480106
2.0     7.488397
2.5     5.503987
1.0     2.787695
1.5     1.776151
0.5     1.358642
Name: proportion, dtype: float64

Conferindo dimensionalidade do conjunto de dados

In [28]:
df_movie_lens.shape

(100836, 4)

## Configurando dataset para o treinamento do modelo

In [ ]:
class MovieDataset:
    '''
    Classe criada com o intuito de ajustar o dataset pandas ao
    treinamento de modelos utilizando o PyTorch, especialmente do que se
    diz respeito à utilização de lotes (batches) durante o treinamento.
    '''
    def __init__(self, users, movies, ratings):
        self.users = users
        self.movies = movies
        self.ratings = ratings

    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, item):

        users = self.users[item]
        movies = self.movies[item]
        ratings = self.ratings[item]

        return {
            "user": torch.tensor(users, dtype=torch.long),
            "movies": torch.tensor(movies, dtype=torch.long),
            "ratings": torch.tensor(ratings, dtype=torch.long),
        }
